In [ ]:
import requests
from playwright.async_api import async_playwright
import asyncio


In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"


async def scrape_fbref(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # or chromium
        context = await browser.new_context()
        page = await context.new_page()

        # Navigate to the page
        await page.goto(url, timeout=60000)

        # Wait until the main content is loaded
        await page.wait_for_selector("div#content", timeout=60000)

        # Get the HTML
        html = await page.content()
        # html = await page.locator("table#stats_table").inner_html()
        await browser.close()
    
    return html



cacheHandler = await scrape_fbref(standings_url)


In [ ]:
data =  cacheHandler
data

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')

In [ ]:
standings_table = soup.select('table.stats_table')[0]

In [ ]:
# assigning all the a tags in the html pages to variable of a links 
links = standings_table.find_all('a')


In [ ]:
links = [l.get("href") for l in links]
links = [l for l in links if '/squad/' in l]

links

In [ ]:
team_urls = [f"https://fbref/com{l}" for l in links]

In [ ]:
team_url = team_urls[0]


In [ ]:
data = requests.get(team_url)

data.text

In [ ]:

matches = pd.reaf_html(data.text, match="Scores & Fixtures")
matches[0]

In [ ]:
soup = BeautifulSoup(data.text)

links = soup.find_all('a')

links = [l.get("href") for l in links]


In [ ]:
links  = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
dt = requests.get(f'https://fbref.com{links[0]}')

In [ ]:
data.text

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]


In [ ]:
shooting.head()

In [ ]:
shooting.columns = shooting.columns.droplevel(0)
shooting.head()


In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")


In [ ]:
shooting.shape()

In [ ]:
years = list(range(2020, 2024), -1)
years


In [ ]:
all_matches = []


In [ ]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
import time

for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.find("table.stats_table")[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squad/' in l]
    team_urls = [f"https://fbref/com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]

        data = requests.get(f"https://fbref.com{links[0]}")

        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel(0)

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)

        time.sleep(1)  # Be polite and avoid overwhelming the server





In [ ]:
match_df = pd.concat(all_matches)


In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]



In [ ]:

match_df.to_csv("matches.csv")

In [ ]:
match_df